# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Оглавление: 
1. [1. Откройте и изучите файл](#step1)  
2. [2. Разбейте данные на выборки](#step2)  
3. [3. Исследуйте модели](#step3)
4. [4. Проверьте модель на тестовой выборке](#step4)
5. [5. (бонус) Проверьте модели на адекватность](#step5)
6. [6. Итоговый вывод](#step6)

## 1. Откройте и изучите файл <a id="step1"></a> 

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')
display(df.head())
print(df.info())
print(df.describe())
print(df.isna().sum())
print("Дубликатов", df.duplicated().sum())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None
             calls      minutes     messages       mb_used     is_ultra
count  3214.000000  3214.000000  3214.000000   3214.000000  3214.000000
mean     63.038892   438.208787    38.281269  17207.673836     0.306472
std      33.236368   234.569872    36.148326   7570.968246     0.461100
min       0.000000     0.000000     0.000000      0.000000     0.000000
25%      40.000000   274.575000     9.000000  12491.902500     0.000000
50%      62.000000   430.600000    30.000000  16943.235000     0.000000
75%      82.000000   571.927500    57.000000  21424.700000     1.000000
max     244.000000  1632.060000   224.000000  49745.730000     1.000000
calls       0
minutes

### Вывод:
Я загрузила таблицу с данными о поведении клиентов тарифов Smart и Ultra. Датасет состоит из 16715 строк и 11 колонок. Пропущенных данных и дублей нет. Типы колонок корректны, можно приступать к дальнейшему анализу. 

## 2. Разбейте данные на выборки <a id="step2"></a> 

In [3]:
#разделяем по правилу: 60-20-20
df_train, df_test = train_test_split(df, test_size=0.4, random_state=123)
df_test, df_valid = train_test_split(df_test, test_size = 0.5, random_state=123)
print("Расмер обучающей выборки:", df_train.shape)
print("Размер валидационной выборки:", df_valid.shape)
print("Размер тестовой выботки:", df_test.shape)

Расмер обучающей выборки: (1928, 5)
Размер валидационной выборки: (643, 5)
Размер тестовой выботки: (643, 5)


In [4]:
def features_target(data, param):
    features = data.drop([param], axis = 1)
    target = data[param]
    return features, target



In [5]:
features_train, target_train = features_target(df_train, 'is_ultra')
features_valid, target_valid = features_target(df_valid, 'is_ultra')
features_test, target_test = features_target(df_test, 'is_ultra')

### Вывод:
Так как у нас нет отдельной тестовой выборки, разбиваем исходные данные на три группы по правилу 3:1:1. Обучающая выборка - 60%, валидационная выборка - 20%, тестовая выборка - 20%. Все данные в выборках я разделина на признаки (features) и целевой признак 'is_ultra' (target).

## 3. Исследуйте модели <a id="step3"></a> 

In [6]:
best_model = None
n_estimatots = 0
max_depth = 0
best_accuracy = 0

In [7]:
# DecisionTreeClassifier
best_model = None
n_estimatots = 0
max_depth = 0
best_accuracy = 0
for i in range(1,11):
    model = DecisionTreeClassifier(random_state = 123, max_depth=i)
    model.fit(features_train, target_train)
    accuracy = model.score(features_valid, target_valid)
    if best_accuracy<accuracy:
        best_model = model
        best_accuracy = accuracy
        max_depth = i
print(best_accuracy)
print(max_depth)

0.7962674961119751
3


In [8]:
# RandomForestClassifier
best_model = None
n_estimatots = 0
max_depth = 0
best_accuracy = 0
for i in range(1,11):
    for j in range(1,51,10):
        model = RandomForestClassifier(random_state = 123, max_depth=i, n_estimators = j)
        model.fit(features_train, target_train)
        accuracy = model.score(features_valid, target_valid)
        if best_accuracy<accuracy:
            best_model = model
            best_accuracy = accuracy
            max_depth = i
            n_estimators = j
print(best_accuracy)
print(max_depth)
print(n_estimators)

0.8055987558320373
6
11


In [9]:
# LogisticRegression 
model = LogisticRegression(random_state = 123)
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)
if best_accuracy<accuracy:
    best_model = model
    best_accuracy = accuracy
print(accuracy)

0.6967340590979783


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [10]:
print("Лучшая модель:", best_model)
print("Точность на валидационной выборке:", best_accuracy)

Лучшая модель: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=6, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=11,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)
Точность на валидационной выборке: 0.8055987558320373


### Вывод:
Я исследовала качество разных моделей, меняя гиперпараметры. Модели обучались на обучающей выборке (features_train, target_train), а их точность проверялась на валидационной выборке (features_valid, target_valid).
В модели решающее дерево (DecisionTreeClassifier) я изменяла гиперпарамерт "глубина дерева" (max_depth), и наилучший результат с точностью 0.796 показала модель с "глубиной дерева" 3.
В модели случайные лес (RandomForestClassifier) я изменяла гиперпарамерт "глубина дерева" (max_depth) и "количество деревьев" (n_estimators). Наилучший результат с точностью 0.8 показала модель с "глубиной дерева" 6  и "количество деревьев" 11.
Точность модели логистическая реграссия (LogisticRegression) равна 0.696.
Самая низкая точность у модели логистической регрессии, а точность моделей решающее дерево и случайный леспрактически одинаковая. Лучше всего себя показала модель решающее дерево со следующими параметрами: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini', max_depth=6, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=11, n_jobs=None, oob_score=False, random_state=123, verbose=0, warm_start=False).

## 4. Проверьте модель на тестовой выборке <a id="step4"></a> 

In [11]:
best_model.fit(features_train, target_train)
accuracy = best_model.score(features_test, target_test)
print("Точность на тестовой выборке:", accuracy)

Точность на тестовой выборке: 0.8102643856920684


### Вывод:
Я проверила лучшую модель (RandomForestClassifier(max_depth=6, n_estimators=11) на тестовой выборке и получила точность 0,81, что является достаточно хорошим показателем.

## 5. (бонус) Проверьте модели на адекватность <a id="step5"></a> 

In [22]:
#проверим адекватность на примере DummyClassifier
dummy_model = DummyClassifier(strategy="most_frequent", random_state=123)
dummy_model.fit(features_train, target_train)
dummy_accuracy = dummy_model.score(features_test, target_test)
print("точность простейшей модели", dummy_accuracy)

0.6982892690513219


### Вывод:
Она представляет собой простейшую (dummy) модель, которая всегда предсказывает наиболее часто встречающийся класс. 

## 6. Итоговый вывод  <a id="step6"></a> 

Я загрузила таблицу с данными о поведении клиентов тарифов Smart и Ultra. Датасет состоит из 16715 строк и 11 колонок. Пропущенных данных и дублей нет. Типы колонок корректны, можно приступать к дальнейшему анализу. 
Так как у нас нет отдельной тестовой выборки, разбиваем исходные данные на три группы по правилу 3:1:1. Обучающая выборка - 60%, валидационная выборка - 20%, тестовая выборка - 20%. Все данные в выборках я разделина на признаки (features) и целевой признак 'is_ultra' (target).
Я исследовала качество разных моделей, меняя гиперпараметры. Модели обучались на обучающей выборке (features_train, target_train), а их точность проверялась на валидационной выборке (features_valid, target_valid).
В модели решающее дерево (DecisionTreeClassifier) я изменяла гиперпарамерт "глубина дерева" (max_depth), и наилучший результат с точностью 0.796 показала модель с "глубиной дерева" 3.
В модели случайные лес (RandomForestClassifier) я изменяла гиперпарамерт "глубина дерева" (max_depth) и "количество деревьев" (n_estimators). Наилучший результат с точностью 0.8 показала модель с "глубиной дерева" 6  и "количество деревьев" 11.
Точность модели логистическая реграссия (LogisticRegression) равна 0.696.
Самая низкая точность у модели логистической регрессии, а точность моделей решающее дерево и случайный лес практически одинаковая. Лучше всего себя показала модель решающее дерево со следующими параметрами: RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini', max_depth=6, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=11, n_jobs=None, oob_score=False, random_state=123, verbose=0, warm_start=False).
Я проверила лучшую модель RandomForestClassifier(max_depth=6, n_estimators=11) на тестовой выборке и получила точность 0,81, что является достаточно хорошим показателем.